In [ ]:
!pip install detoxify

import pandas as pd
from detoxify import Detoxify
from transformers import AutoTokenizer
from huggingface_hub import login
from tqdm import tqdm

def to_python_float(value):
    """
    Convertit en float Python (en gérant le None).
    """
    return float(value) if value is not None else None

def get_detoxify_pretrained_name(model_variant: str) -> str:
    """
    Associe le nom du modèle Hugging Face à un 'model_variant' Detoxify.
    En effet, Detoxify('unbiased') utilise en général 'unitary/unbiased-toxic-roberta'.
    """
    mapping = {
        'unbiased': 'unitary/unbiased-toxic-roberta',
        'original': 'unitary/toxic-bert',
        'multilingual': 'unitary/multilingual-toxic-xlm-roberta'
    }
    return mapping.get(model_variant, 'unitary/unbiased-toxic-roberta')

def analyze_with_sliding_window_tokens(
    text: str,
    toxicity_model,
    tokenizer,
    window_size_tokens=512,
    stride_tokens=256
):
    """
    Analyse la toxicité avec Detoxify en procédant par découpe en tokens (sliding window).
    """
    all_ids = tokenizer.encode(text, add_special_tokens=False)
    n_tokens = len(all_ids)

    results = []
    start_idx = 0

    while start_idx < n_tokens:
        end_idx = min(start_idx + window_size_tokens, n_tokens)
        chunk_ids = all_ids[start_idx:end_idx]

        # Décoder le chunk
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        # Prédiction Detoxify
        chunk_scores = toxicity_model.predict(chunk_text)

        results.append({
            "start_token_idx": start_idx,
            "end_token_idx": end_idx,
            "scores": chunk_scores
        })

        start_idx += stride_tokens

    return results

def process_and_store_toxicities_token_based_csv(
    input_csv,
    output_csv,
    model_variant="unbiased",  # 'original', 'unbiased', ou 'multilingual'
    huggingface_token=None,    # <--- On peut passer ici le token Hugging Face si nécessaire
    window_size=512,
    stride=256
):
    """
    1) Lit un CSV dont chaque ligne contient au moins une colonne 'text'.
    2) Applique l’analyse par fenêtres glissantes *en tokens* via Detoxify sur chaque texte.
    3) Calcule pour chaque ligne les scores moyens pour toutes les dimensions de toxicité.
    4) Réécrit le même CSV en ajoutant les colonnes 'toxicity_*'.
    """

    # -- Si un token est fourni, on se log sur Hugging Face avant de charger le modèle --
    if huggingface_token:
        login(token=huggingface_token)

    # -- Charger le modèle Detoxify --
    toxicity_model = Detoxify(model_variant)

    # -- Récupérer le nom du tokenizer --
    pretrained_name = get_detoxify_pretrained_name(model_variant)

    # -- Charger le tokenizer en passant éventuellement le token si nécessaire --
    tokenizer = AutoTokenizer.from_pretrained(pretrained_name, use_auth_token=huggingface_token)

    # -- Charger le CSV --
    df = pd.read_csv(input_csv)

    # Préparer les colonnes à ajouter
    toxicity_labels = [
        "toxicity",
        "severe_toxicity",
        "obscene",
        "identity_attack",
        "insult",
        "threat",
        "sexual_explicit",
    ]
    for label in toxicity_labels:
        df[f"toxicity_{label}"] = None

    # Parcourir chaque ligne du CSV
    for i in tqdm(range(len(df)), desc="Processing rows"):
        text = df.loc[i, 'text']
        if not isinstance(text, str) or not text.strip():
            continue

        # Analyse par fenêtres glissantes
        results = analyze_with_sliding_window_tokens(
            text=text,
            toxicity_model=toxicity_model,
            tokenizer=tokenizer,
            window_size_tokens=window_size,
            stride_tokens=stride
        )

        # On regroupe les résultats (moyenne par défaut)
        all_scores = {label: [] for label in toxicity_labels}
        for res in results:
            scores = res["scores"]
            for label in toxicity_labels:
                val = to_python_float(scores.get(label))
                if val is not None:
                    all_scores[label].append(val)

        avg_scores = {}
        for label in toxicity_labels:
            values = all_scores[label]
            avg_scores[label] = sum(values) / len(values) if values else None

        # Stocker la moyenne dans le DataFrame
        for label in toxicity_labels:
            df.loc[i, f"toxicity_{label}"] = avg_scores[label]

    # -- Sauvegarder le CSV modifié --
    df.to_csv(output_csv, index=False)
    print(f"CSV réécrit avec colonnes de toxicité ajoutées : {output_csv}")

process_and_store_toxicities_token_based_csv(
    input_csv="input.csv",      # Remplacez par le chemin de votre CSV d'entrée
    output_csv="output.csv",    # Remplacez par le chemin du CSV de sortie
    model_variant="unbiased",   # Options : 'unbiased', 'original', 'multilingual'
    huggingface_token=""  # Remplacez par votre token Hugging Face si nécessaire
)